<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/rossmann_timeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://colab.research.google.com/github/duchaba2/fastai_san_ramon_biztech/blob/master/smbt_rossman_data_clean.ipynb#scrollTo=UImWYEGiaFUS
# https://www.kaggle.com/hortonhearsafoo/fast-ai-lesson-3

In [ ]:
# Last amended: 17th March, 2021
# My folder: 
# Objectives:
#           i) Predicting sales in Rossmann Store Sales
#          ii) Feature generation in TimeSeries data

# The problem
Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### Field descriptions

Most of the fields are self-explanatory. The following are descriptions for those that aren't.
    
>  **Id** - an Id that represents a (Store, Date) duple within the test set<br>
>  **Store** - a unique Id for each store<br>
>  **Sales** - the turnover for any given day (this is what you are predicting)<br>
>  **Customers** - the number of customers on a given day<br>
>  **Open** - an indicator for whether the store was open: 0 = closed, 1 = open<br>
>  **StateHoliday** - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None<br>
>  **SchoolHoliday** - indicates if the (Store, Date) was affected by the closure of public schools<br>
>  **StoreType** - differentiates between 4 different store models: a, b, c, d<br>
>  **Assortment** - describes an assortment level: a = basic, b = extra, c = extended<br>
>   **CompetitionDistance** - distance in meters to the nearest competitor store<br>
>  **CompetitionOpenSince**[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened<br>
>  **Promo** - indicates whether a store is running a promo on that day<br>
> **Promo2** - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating<br>
> **Promo2Since**[Year/Week] - describes the year and calendar week when the store started participating in Promo2<br>
>  **PromoInterval** - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store


In [ ]:
#import pytorch and AI
#!pip install --upgrade git+https://github.com/fastai/fastai.git

In [ ]:
#(Optional) double check your import pytorch and fast.ai version
# fastai related
import fastai
import fastai.tabular
from fastai.torch_basics import *
from fastai.data.all import *
#fastai.utils.show_install()

In [1]:
# 1.0 connect to your google drive
#     Transfer files from gdrive to colab VM

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 1.1 Copy files from source to Colab
source="/content/drive/MyDrive/Colab_data_files/rossmannStoreSales"
dest="/content"

! rm -rf $dest/rossmannStoreSales
! cp -r $source $dest/rossmannStoreSales
! ls -la $dest/rossmannStoreSales


total 13512
drwx------ 2 root root    4096 Mar 17 00:37 .
drwxr-xr-x 1 root root    4096 Mar 17 00:37 ..
-rw------- 1 root root 7730448 Mar 17 00:37 rossmann.tgz
-rw------- 1 root root    8529 Mar 17 00:37 store.csv.zip
-rw------- 1 root root  146686 Mar 17 00:37 test.csv.zip
-rw------- 1 root root 5931096 Mar 17 00:37 train.csv.zip


In [ ]:
# 1.2 Untar tgz file
! tar -xvzf $dest/rossmannStoreSales/rossmann.tgz -C $dest/rossmannStoreSales/
! ls -la $dest/rossmannStoreSales

In [5]:
# 1.3 Call libraries
import pandas as pd
import numpy as np
import os

In [6]:
# 1.4
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
# 1.5 
path = "/content/rossmannStoreSales"
os.chdir(path)
os.listdir()

['test.csv.zip',
 'store.csv.zip',
 'store_states.csv',
 'rossmann.tgz',
 'state_names.csv',
 'googletrend.csv',
 'sample_submission.csv',
 'store.csv',
 'weather.csv',
 'test.csv',
 'train.csv.zip',
 'train.csv']

In [10]:
# 2.0 Read files
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
store = pd.read_csv("store.csv")
googletrend = pd.read_csv("googletrend.csv")
weather = pd.read_csv("weather.csv")
state_names = pd.read_csv("state_names.csv")
store_states = pd.read_csv("store_states.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
# 2.1 Look at train data
print("\n---train----\n")
train.shape         # (1017209, 9)
print("\n------train------\n")
train.head()
print("\n-----Summary------\n")
train.describe()
print("\n-----dtypes------\n")
train.dtypes


---train----



(1017209, 9)


------train------



,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1



-----Summary------



,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00



-----dtypes------



Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

In [ ]:
# 2.2
train['Store'].nunique()             # 1115
print()
train['Promo'].nunique()             # 2
train['Promo'].unique()              # [1,0] 
print()
train['Open'].nunique()              # 2
train['Open'].unique()               # [1,0]
print()
train['SchoolHoliday'].nunique()     # 2
train['SchoolHoliday'].unique()
print()
train['StateHoliday'].nunique()      # 5
train['StateHoliday'].unique()       # ['0', 'a', 'b', 'c', 0]

In [41]:
# 2.3 About nulls
train.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [27]:
# 3.0 Look at store data
print("\n---shape----\n")
store.shape         # (1017209, 9)
print("\n------data------\n")
store.head()
print("\n-----Summary------\n")
store.describe()
print("\n-----dtypes------\n")
store.dtypes


---shape----



(1115, 10)


------data------



,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN



-----Summary------



,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000



-----dtypes------



Store                          int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [30]:
# 3.1
store['StoreType'].unique()     # ['c', 'a', 'd', 'b']
store['Assortment'].unique()    # ['a', 'c', 'b']
store['Promo2'].unique()        # [0,1]
store['CompetitionOpenSinceYear'].unique()  # Large number of years from 1900 to 2015

array(['c', 'a', 'd', 'b'], dtype=object)

array(['a', 'c', 'b'], dtype=object)

array([0, 1])

array([2008., 2007., 2006., 2009., 2015., 2013., 2014., 2000., 2011.,
         nan, 2010., 2005., 1999., 2003., 2012., 2004., 2002., 1961.,
       1995., 2001., 1990., 1994., 1900., 1998.])

In [42]:
# 3.2 Whenever Promo2 is zero, three other columns carry NaN values
store.loc[store['Promo2'] == 0, ['Promo2','Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']].head(5)
print()
store.isnull().sum()

,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,NaN,NaN,NaN
3,0,NaN,NaN,NaN
4,0,NaN,NaN,NaN
5,0,NaN,NaN,NaN
6,0,NaN,NaN,NaN


Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [103]:
# Look at train data
print("\n---train----\n")
train.shape         # (1017209, 9)
print("\n------test------\n")
test.shape          # (41088, 8)
print("\n-----store------\n")
store.shape         # (1115, 10)
print("\n------state_names-------\n")
state_names.shape   # (16,2)
print("\n------store_states------\n")
store_states.shape  # (1115,2)
print("\n------googletrend-------\n")
googletrend.shape   # (2072,3)
print("\n-------weather------\n")
weather.shape       # (15840, 24)


---train----



(1017209, 9)


------test------



(41088, 8)


-----store------



...

(1115, 2)


------googletrend-------



(2072, 3)


-------weather------



(15840, 24)

In [104]:
# Look at it
print("\n---train----\n")
train.head(5)
print("\n------test------\n")
test.head(5)
print("\n-----store------\n")
store.head(5)
print("\n------state_names-------\n")
state_names.tail(5)
print("\n------store_states------\n")
store_states.head(5)
print("\n------googletrend-------\n")
googletrend.head(5)
print("\n-------weather------\n")
weather.tail(5)


---train----



,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1



------test------



,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0



-----store------



...

,Store,State
0,1,HE
1,2,TH
2,3,NW
3,4,BE
4,5,SN



------googletrend-------



,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67



-------weather------



,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees
15835,Saarland,2015-09-13,21,17,12,16,14,12,100,88,68,1015,1010,1005,31.0,15.0,10.0,27,10,50.0,1.02,7.0,Rain,113
15836,Saarland,2015-09-14,18,14,11,15,12,7,99,85,61,1009,1005,1004,31.0,13.0,4.0,32,16,53.0,9.91,5.0,Rain-Thunderstorm,213
15837,Saarland,2015-09-15,16,12,9,11,8,7,93,77,62,1010,1008,1004,31.0,12.0,10.0,34,14,NaN,0.00,5.0,Rain,193
15838,Saarland,2015-09-16,19,15,11,16,13,10,97,90,75,1004,999,995,31.0,10.0,4.0,32,14,45.0,20.07,7.0,Rain-Thunderstorm,147
15839,Saarland,2015-09-17,14,13,12,14,12,10,99,92,82,1013,1005,999,31.0,14.0,8.0,27,16,47.0,6.10,6.0,Rain,202


In [105]:
train.StateHoliday.dtype
train.StateHoliday[:4]

dtype('O')

0    0
1    0
2    0
3    0
Name: StateHoliday, dtype: object

In [106]:
train.StateHoliday = train.StateHoliday!='0'
test.StateHoliday = test.StateHoliday!='0'

In [107]:
train.StateHoliday[:4]

0    False
1    False
2    False
3    False
Name: StateHoliday, dtype: bool

join_df is a function for joining tables on specific fields. By default, we'll be doing a left outer join of right on the left argument using the given fields for each table.

Pandas does joins using the merge method. The suffixes argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "_y" to those on the right.


In [108]:
# df.merge(right,
#          how='inner',
#          left_on=None,     # Column or index level names to join on in the left DataFrame
#          right_on=None,    # Column or index level names to join on in the right DataFrame.
#          suffixes=('_x', '_y')) #  a string indicating the suffix to add to overlapping column name

def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [109]:
weather = join_df(weather,     # Left df
                  state_names, # right df
                  "file",      # Ist column of 'weather'
                  "StateName"  # Ist column of 'state_name'
                  )

In [110]:
weather.shape   # (15840, 24 + 2)
weather.head(2) # Both 'file' and 'StateName' have same data


(15840, 26)

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,65,1013,1005,1001,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,78,1028,1022,1014,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW


In [111]:
googletrend.head(10)

,file,week,trend
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67
5,Rossmann_DE_SN,2013-01-06 - 2013-01-12,71
6,Rossmann_DE_SN,2013-01-13 - 2013-01-19,53
7,Rossmann_DE_SN,2013-01-20 - 2013-01-26,58
8,Rossmann_DE_SN,2013-01-27 - 2013-02-02,55
9,Rossmann_DE_SN,2013-02-03 - 2013-02-09,55


In [112]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html
googletrend.week.head(2)
print()
googletrend.week.str.split(' - ').head(2)                     # split strings into a list
print()
googletrend.week.str.split(' - ', expand = True).head(2)      # split strings into separate columns
print()
googletrend.week.str.split(' - ', expand = True)[0].head(2)   # select the first split column


0    2012-12-02 - 2012-12-08
1    2012-12-09 - 2012-12-15
Name: week, dtype: object

0    [2012-12-02, 2012-12-08]
1    [2012-12-09, 2012-12-15]
Name: week, dtype: object

,0,1
0,2012-12-02,2012-12-08
1,2012-12-09,2012-12-15


0    2012-12-02
1    2012-12-09
Name: 0, dtype: object

In [ ]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]


In [ ]:
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]


In [ ]:
np.sum(googletrend.State=='NI')

148

In [ ]:
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'

In [113]:
googletrend['Date'] = googletrend.week.str.split(' - ', expand=True)[0]
googletrend['State'] = googletrend.file.str.split('_', expand=True)[2]
googletrend.loc[googletrend.State=='NI', "State"] = 'HB,NI'
googletrend.head()

,file,week,trend,Date,State
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN


In [ ]:
#from fastai import *
#from fastai.tabular import *

In [114]:
import re
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [115]:
add_datepart(weather, "Date", drop=False)
add_datepart(googletrend, "Date", drop=False)
add_datepart(train, "Date", drop=False)
add_datepart(test, "Date", drop=False)
add_datepart(weather, "Date", drop=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  from ipykernel import kernelapp as app


In [116]:
weather.shape    # (15840, 39)
weather.head()
googletrend.head()
train.head()
test.head()



(15840, 39)

,file,Date,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,NordrheinWestfalen,2013-01-01,8,4,2,7,5,1,94,87,65,1013,1005,1001,31.0,12.0,4.0,39,26,58.0,5.08,6.0,Rain,215,NordrheinWestfalen,NW,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400
1,NordrheinWestfalen,2013-01-02,7,4,1,5,3,2,93,85,78,1028,1022,1014,31.0,14.0,10.0,24,16,NaN,0.00,6.0,Rain,225,NordrheinWestfalen,NW,2013,1,1,2,2,2,False,False,False,False,False,False,1357084800
2,NordrheinWestfalen,2013-01-03,11,8,6,10,8,4,100,93,77,1035,1030,1026,31.0,8.0,2.0,26,21,NaN,1.02,7.0,Rain,240,NordrheinWestfalen,NW,2013,1,1,3,3,3,False,False,False,False,False,False,1357171200
3,NordrheinWestfalen,2013-01-04,9,9,8,9,9,8,100,94,87,1036,1035,1034,11.0,5.0,2.0,23,14,NaN,0.25,7.0,Rain,263,NordrheinWestfalen,NW,2013,1,1,4,4,4,False,False,False,False,False,False,1357257600
4,NordrheinWestfalen,2013-01-05,8,8,7,8,7,6,100,94,87,1035,1034,1033,10.0,6.0,3.0,16,10,NaN,0.00,7.0,Rain,268,NordrheinWestfalen,NW,2013,1,1,5,5,5,False,False,False,False,False,False,1357344000


,file,week,trend,Date,State,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,Rossmann_DE_SN,2012-12-02 - 2012-12-08,96,2012-12-02,SN,2012,12,48,2,6,337,False,False,False,False,False,False,1354406400
1,Rossmann_DE_SN,2012-12-09 - 2012-12-15,95,2012-12-09,SN,2012,12,49,9,6,344,False,False,False,False,False,False,1355011200
2,Rossmann_DE_SN,2012-12-16 - 2012-12-22,91,2012-12-16,SN,2012,12,50,16,6,351,False,False,False,False,False,False,1355616000
3,Rossmann_DE_SN,2012-12-23 - 2012-12-29,48,2012-12-23,SN,2012,12,51,23,6,358,False,False,False,False,False,False,1356220800
4,Rossmann_DE_SN,2012-12-30 - 2013-01-05,67,2012-12-30,SN,2012,12,52,30,6,365,False,False,False,False,False,False,1356825600


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1,5,2015-07-31,5263,555,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
1,2,5,2015-07-31,6064,625,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
2,3,5,2015-07-31,8314,821,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
3,4,5,2015-07-31,13995,1498,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800
4,5,5,2015-07-31,4822,559,1,1,False,1,2015,7,31,31,4,212,True,False,False,False,False,False,1438300800


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,1,1,4,2015-09-17,1.0,1,False,0,2015,9,38,17,3,260,False,False,False,False,False,False,1442448000
1,2,3,4,2015-09-17,1.0,1,False,0,2015,9,38,17,3,260,False,False,False,False,False,False,1442448000
2,3,7,4,2015-09-17,1.0,1,False,0,2015,9,38,17,3,260,False,False,False,False,False,False,1442448000
3,4,8,4,2015-09-17,1.0,1,False,0,2015,9,38,17,3,260,False,False,False,False,False,False,1442448000
4,5,9,4,2015-09-17,1.0,1,False,0,2015,9,38,17,3,260,False,False,False,False,False,False,1442448000


In [ ]:
#import fastai
#from fastai.tabular import *
#from fastai.column_data import *

In [117]:
# For whole of Germany. instead of individual States
trend_de = googletrend[googletrend.file == 'Rossmann_DE']

In [118]:
trend_de.shape      # (148, 18)
googletrend.shape   # (2072, 18)

(148, 18)

(2072, 18)

In [119]:
store = join_df(store, store_states, "Store")
len(store[store.State.isnull()])

0

In [120]:
joined = join_df(train, store, "Store")
joined_test = join_df(test, store, "Store")
len(joined[joined.StoreType.isnull()]),len(joined_test[joined_test.StoreType.isnull()])

(0, 0)

In [121]:
joined = join_df(joined, googletrend, ["State","Year", "Week"])
joined_test = join_df(joined_test, googletrend, ["State","Year", "Week"])
len(joined[joined.trend.isnull()]),len(joined_test[joined_test.trend.isnull()])

(0, 0)

In [122]:
joined = joined.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
joined_test = joined_test.merge(trend_de, 'left', ["Year", "Week"], suffixes=('', '_DE'))
len(joined[joined.trend_DE.isnull()]),len(joined_test[joined_test.trend_DE.isnull()])

(0, 0)

In [123]:
joined = join_df(joined, weather, ["State","Date"])
joined_test = join_df(joined_test, weather, ["State","Date"])
len(joined[joined.Mean_TemperatureC.isnull()]),len(joined_test[joined_test.Mean_TemperatureC.isnull()])

(0, 0)

In [124]:
joined.columns
joined.name = "joined"
joined_test.name = "joined_test"

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed', 'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'State', 'file',
       'week', 'trend', 'Date_y', 'Month_y', 'Day_y', 'Dayofweek_y',
       'Dayofyear_y', 'Is_month_end_y', 'Is_month_start_y', 'Is_quarter_end_y',
       'Is_quarter_start_y', 'Is_year_end_y', 'Is_year_start_y', 'Elapsed_y',
       'file_DE', 'week_DE', 'trend_DE', 'Date_DE', 'State_DE', 'Month_DE',
       'Day_DE', 'Dayofweek_DE', 'Dayofyear_DE', 'Is_month_end_DE',
       'Is_month_start_DE', 'Is_quarter_end_DE', 'Is_quarter_start_DE',
       'Is_year_end_DE', 'Is_year_start_DE',

In [125]:
for df in (joined, joined_test):
    for c in df.columns:
        if c.endswith('_y'):
            if c in df.columns:
              print(c,df.name)
              df.drop(c, inplace=True, axis=1)

Date_y joined
Month_y joined
Day_y joined
Dayofweek_y joined
Dayofyear_y joined
Is_month_end_y joined
Is_month_start_y joined
Is_quarter_end_y joined
Is_quarter_start_y joined
Is_year_end_y joined
Is_year_start_y joined
Elapsed_y joined
file_y joined
Year_y joined
Week_y joined
Date_y joined_test
Month_y joined_test
Day_y joined_test
Dayofweek_y joined_test
Dayofyear_y joined_test
Is_month_end_y joined_test
Is_month_start_y joined_test
Is_quarter_end_y joined_test
Is_quarter_start_y joined_test
Is_year_end_y joined_test
Is_year_start_y joined_test
Elapsed_y joined_test
file_y joined_test
Year_y joined_test
Week_y joined_test


In [126]:
joined[['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'Promo2SinceWeek']].head(20)
joined[['CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'Promo2SinceWeek']].describe()

,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,Promo2SinceYear,Promo2SinceWeek
0,2008.0,9.0,NaN,NaN
1,2007.0,11.0,2010.0,13.0
2,2006.0,12.0,2011.0,14.0
3,2009.0,9.0,NaN,NaN
4,2015.0,4.0,NaN,NaN
5,2013.0,12.0,NaN,NaN
6,2013.0,4.0,NaN,NaN
7,2014.0,10.0,NaN,NaN
8,2000.0,8.0,NaN,NaN
9,2009.0,9.0,NaN,NaN


,CompetitionOpenSinceYear,CompetitionOpenSinceMonth,Promo2SinceYear,Promo2SinceWeek
count,693861.000000,693861.000000,509178.000000,509178.000000
mean,2008.690228,7.222866,2011.752774,23.269093
std,5.992644,3.211832,1.662870,14.095973
min,1900.000000,1.000000,2009.000000,1.000000
25%,2006.000000,4.000000,2011.000000,13.000000
50%,2010.000000,8.000000,2012.000000,22.000000
75%,2013.000000,10.000000,2013.000000,37.000000
max,2015.000000,12.000000,2015.000000,50.000000


In [127]:
for df in (joined,joined_test):
    df['CompetitionOpenSinceYear'] = df.CompetitionOpenSinceYear.fillna(1900).astype(np.int32)
    df['CompetitionOpenSinceMonth'] = df.CompetitionOpenSinceMonth.fillna(1).astype(np.int32)
    df['Promo2SinceYear'] = df.Promo2SinceYear.fillna(1900).astype(np.int32)
    df['Promo2SinceWeek'] = df.Promo2SinceWeek.fillna(1).astype(np.int32)

In [128]:
joined[['Date','CompetitionOpenSinceYear','CompetitionOpenSinceMonth'] ]

,Date,CompetitionOpenSinceYear,CompetitionOpenSinceMonth
0,2015-07-31,2008,9
1,2015-07-31,2007,11
2,2015-07-31,2006,12
3,2015-07-31,2009,9
4,2015-07-31,2015,4
...,...,...,...
1017204,2013-01-01,2014,6
1017205,2013-01-01,2006,4
1017206,2013-01-01,1900,1
1017207,2013-01-01,1900,1


In [129]:
for df in (joined,joined_test):
    df["CompetitionOpenSince"] = pd.to_datetime(dict(year=df.CompetitionOpenSinceYear, 
                                                     month=df.CompetitionOpenSinceMonth, day=15))
    df["CompetitionDaysOpen"] = df.Date.subtract(df.CompetitionOpenSince).dt.days

In [130]:
for df in (joined,joined_test):
    df.loc[df.CompetitionDaysOpen<0, "CompetitionDaysOpen"] = 0
    df.loc[df.CompetitionOpenSinceYear<1990, "CompetitionDaysOpen"] = 0

In [131]:
joined.shape   # (1017209, 76)

(1017209, 76)

In [ ]:
#export
def add_elapsed_times(df, field_names, date_field, base_field):
    "Add in `df` for each event in `field_names` the elapsed time according to `date_field` grouped by `base_field`"
    field_names = list(L(field_names))
    #Make sure date_field is a date and base_field a bool
    df[field_names] = df[field_names].astype('bool')
    make_date(df, date_field)

    work_df = df[field_names + [date_field, base_field]]
    work_df = work_df.sort_values([base_field, date_field])
    work_df = _get_elapsed(work_df, field_names, date_field, base_field, 'After')
    work_df = work_df.sort_values([base_field, date_field], ascending=[True, False])
    work_df = _get_elapsed(work_df, field_names, date_field, base_field, 'Before')

    for a in ['After' + f for f in field_names] + ['Before' + f for f in field_names]:
        work_df[a] = work_df[a].fillna(0).astype(int)

    for a,s in zip([True, False], ['_bw', '_fw']):
        work_df = work_df.set_index(date_field)
        tmp = (work_df[[base_field] + field_names].sort_index(ascending=a)
                      .groupby(base_field).rolling(7, min_periods=1).sum())
        tmp.drop(base_field,1,inplace=True)
        tmp.reset_index(inplace=True)
        work_df.reset_index(inplace=True)
        work_df = work_df.merge(tmp, 'left', [date_field, base_field], suffixes=['', s])
    work_df.drop(field_names,1,inplace=True)
    return df.merge(work_df, 'left', [date_field, base_field])

```
# Initialize
last_store_Seen    = 0
last_date_recorded = np.datetime64() 

1. Read current store number: csn
2. Read current SchoolHoliday value: sh_value
3. Read current Date: c_date
Begin  
Is csn == last_store_seen
  if NO:   
          last_store_seen = csn
          after = 0
          is sh_value == True
            if Yes, last_date_recorded = c_date
            if No---last_date_recorded = np.datetime64() 
   if YES:
          is sh_value == True
            if Yes, last_date_recorded = c_date
            after = 0
            if False, after = c_date - last_date_recorded

```





In [ ]:
df = joined.copy()
df = df.sort_values(['Store', 'Date'])
out = get_elapsed('Store', 'SchoolHoliday', 'Date', df)
out1 = get_myelapsed('Store', 'SchoolHoliday', 'Date', df)
np.nansum(out)   # 23758783.0
print()
np.nansum(out1)  # 23758783.0

In [150]:
out1

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 numpy.timedelta64(1,'D'),
 numpy.timedelta64(2,'D'),
 numpy.timedelta64(3,'D'),
 numpy.timedelta64(4,'D'),
 numpy.timedelta64(5,'D'),
 numpy.timedelta64(6,'D'),
 numpy.timedelta64(7,'D'),
 numpy.timedelta64(8,'D'),
 numpy.timedelta64(9,'D'),
 numpy.timedelta64(10,'D'),
 numpy.timedelta64(11,'D'),
 numpy.timedelta64(12,'D'),
 numpy.timedelta64(13,'D'),
 numpy.timedelta64(14,'D'),
 numpy.timedelta64(15,'D'),
 numpy.timedelta64(16,'D'),
 numpy.timedelta64(17,'D'),
 numpy.timedelta64(18,'D'),
 numpy.timedelta64(19,'D'),
 numpy.timedelta64(20,'D'),
 numpy.timedelta64(21,'D'),
 numpy.timedelta64(22,'D'),
 numpy.timedelta64(23,'D'),
 numpy.timedelta64(24,'D'),
 numpy.timedelta64(25,'D'),
 numpy.timedelta64(26,'D'),
 numpy.timedelta64(27,'D'),
 numpy.timedelta64(28,'D'),
 numpy.timedelta64(29,'D'),
 numpy.timedelta64(30,'D'),
 numpy.timedelta64(31,'D'),
 numpy.timedelta64(32,'D'),
 numpy.timedelta64(33,'D'),
 numpy.timedelta64(34,'D'),
 numpy.timede

In [147]:
df = df.sort_values(['Store', 'Date'], ascending=[True, False])
out = get_elapsed('Store', 'SchoolHoliday', 'Date', df)
out1 = get_myelapsed('Store', 'SchoolHoliday', 'Date', df)
np.nansum(out)   # -23802703.0
print()
np.nansum(out1)  # -23802703.0

-23802703.0

-23802703.0

In [148]:
# See StackOverflow
#     https://stackoverflow.com/a/18215499/3282777

def get_event_elapsed(grfld, event_fld, pre,df):
  """
  Example:
   grfld = 'Store'
   fld = 'SchoolHoliday'
   pre = 'After'
   So for a given 'Store', next 'SchoolHoliday'
   has come After how many days since the last
  """
  day1 = np.timedelta64(1, 'D')
  last_store_seen = 0
  last_date_recorded = np.datetime64()  # Nat: Not a time
  after = 0
  res = []
  for csn,sh_value,c_date in zip(df[grfld].values,df[fld].values,df[pre].values):
    # Get current store
    if csn != last_store_seen:
      after = 0
      last_store_seen = csn
      if sh_value:
        last_date_recorded = c_date
      else:
        last_date_recorded = np.datetime64()
    else:
      if sh_value:
        last_date_recorded = c_date
        after =0
      else:
        """
        StackOverFlow: https://stackoverflow.com/a/18215499/3282777
        In the absence of division by day1
        we get the following: 
        numpy.timedelta64(1,'D'),
        numpy.timedelta64(2,'D'),
        numpy.timedelta64(3,'D'),
        numpy.timedelta64(4,'D'),
        """
        after = (c_date - last_date_recorded).astype('timedelta64[D]') /  day1
    res.append(after)
  return(res)    
  


In [133]:
def get_elapsed(grpfld,fld, pre, df):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df[grpfld].values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    return res
    

In [99]:
def get_elapsed(grpfld,fld, pre, df):
    day1 = np.timedelta64(1, 'D')
    last_date = np.datetime64()
    last_store = 0
    res = []

    for s,v,d in zip(df[grpfld].values,df[fld].values, df.Date.values):
        if s != last_store:
            last_date = np.datetime64()
            last_store = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res
    return df

In [ ]:
day1 = np.timedelta64(1, 'D')
day1
print()
np.datetime64()

numpy.timedelta64(1,'D')

numpy.datetime64('NaT')

In [ ]:
pd.DataFrame(event = [1,0,1,1,0,1, 0,0,0,0,1,1, 0,0,1,0], 
             Date = ['01/03/2021', '02/03/2021', '03/03/2021', '04/03/2021', '05/03/2021', '06/03/2021',
                     '07/03/2021', '08/03/2021', '09/03/2021', '10/03/2021', '11/03/2021', '12/03/2021',
                     '13/03/2021', '14/03/2021', '15/03/2021', '16/03/2021'],
             base_field = )

In [154]:
columns = ['SchoolHoliday', 'StateHoliday', 'Promo']

In [152]:
df = df.set_index("Date")

In [153]:
df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,State,file,week,trend,file_DE,week_DE,trend_DE,Date_DE,State_DE,Month_DE,Day_DE,Dayofweek_DE,Dayofyear_DE,Is_month_end_DE,Is_month_start_DE,Is_quarter_end_DE,Is_quarter_start_DE,Is_year_end_DE,Is_year_start_DE,Elapsed_DE,Max_TemperatureC,Mean_TemperatureC,Min_TemperatureC,Dew_PointC,MeanDew_PointC,Min_DewpointC,Max_Humidity,Mean_Humidity,Min_Humidity,Max_Sea_Level_PressurehPa,Mean_Sea_Level_PressurehPa,Min_Sea_Level_PressurehPa,Max_VisibilityKm,Mean_VisibilityKm,Min_VisibilitykM,Max_Wind_SpeedKm_h,Mean_Wind_SpeedKm_h,Max_Gust_SpeedKm_h,Precipitationmm,CloudCover,Events,WindDirDegrees,StateName,CompetitionOpenSince,CompetitionDaysOpen,DateSchoolHoliday
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,True,1,2013,1,1,1,1,1,False,True,False,True,False,True,1356998400,c,a,1270.0,9,2008,0,1,1900,NaN,HE,Rossmann_DE_HE,2013-01-06 - 2013-01-12,73,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,1,6,6,6,False,False,False,False,False,False,1357430400,8,6,3,6,3,1,93,80,59,1015,1008,1006,31.0,12.0,10.0,23,14,39.0,2.03,6.0,Rain,206,Hessen,2008-09-15,1569,0.0
2013-01-02,1,3,5530,668,1,0,False,1,2013,1,1,2,2,2,False,False,False,False,False,False,1357084800,c,a,1270.0,9,2008,0,1,1900,NaN,HE,Rossmann_DE_HE,2013-01-06 - 2013-01-12,73,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,1,6,6,6,False,False,False,False,False,False,1357430400,6,3,0,3,1,-1,100,85,67,1031,1024,1015,31.0,13.0,10.0,19,11,37.0,0.00,6.0,NaN,200,Hessen,2008-09-15,1570,0.0
2013-01-03,1,4,4327,578,1,0,False,1,2013,1,1,3,3,3,False,False,False,False,False,False,1357171200,c,a,1270.0,9,2008,0,1,1900,NaN,HE,Rossmann_DE_HE,2013-01-06 - 2013-01-12,73,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,1,6,6,6,False,False,False,False,False,False,1357430400,7,4,3,7,3,1,100,90,77,1036,1032,1030,19.0,10.0,6.0,29,19,47.0,0.00,7.0,Rain,191,Hessen,2008-09-15,1571,0.0
2013-01-04,1,5,4486,619,1,0,False,1,2013,1,1,4,4,4,False,False,False,False,False,False,1357257600,c,a,1270.0,9,2008,0,1,1900,NaN,HE,Rossmann_DE_HE,2013-01-06 - 2013-01-12,73,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,1,6,6,6,False,False,False,False,False,False,1357430400,11,9,6,9,8,6,100,93,79,1036,1034,1033,31.0,11.0,6.0,21,13,37.0,0.00,6.0,Rain,210,Hessen,2008-09-15,1572,0.0
2013-01-05,1,6,4997,635,1,0,False,1,2013,1,1,5,5,5,False,False,False,False,False,False,1357344000,c,a,1270.0,9,2008,0,1,1900,NaN,HE,Rossmann_DE_HE,2013-01-06 - 2013-01-12,73,Rossmann_DE,2013-01-06 - 2013-01-12,62,2013-01-06,None,1,6,6,6,False,False,False,False,False,False,1357430400,9,8,7,8,6,5,87,78,67,1033,1032,1032,31.0,12.0,10.0,24,18,NaN,0.00,7.0,Rain,281,Hessen,2008-09-15,1573,0.0


In [155]:
bwd = df[['Store']+columns].sort_index().groupby("Store").rolling(7, min_periods=1).sum()

In [156]:
bwd.head()

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                 
1     2013-01-01    1.0            1.0           1.0    0.0
      2013-01-02    2.0            2.0           1.0    0.0
      2013-01-03    3.0            3.0           1.0    0.0
      2013-01-04    4.0            4.0           1.0    0.0
      2013-01-05    5.0            5.0           1.0    0.0

In [157]:
fwd = df[['Store']+columns].sort_index(ascending=False
                                      ).groupby("Store").rolling(7, min_periods=1).sum()

In [158]:
fwd.head()

Store  SchoolHoliday  StateHoliday  Promo
Store Date                                                 
1     2015-07-31    1.0            1.0           0.0    1.0
      2015-07-30    2.0            2.0           0.0    2.0
      2015-07-29    3.0            3.0           0.0    3.0
      2015-07-28    4.0            4.0           0.0    4.0
      2015-07-27    5.0            5.0           0.0    5.0